In [1]:
#2018-08-09 07:52:31.286127
%load_ext metapack.jupyter.magic

In [2]:
CACHE_DIR='/Users/eric/Library/Application Support/metapack/'
RESOURCE_NAME='tides_river_rain'
RESOLVED_URL='file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-water_quality/notebooks/Combine.ipynb#comb'
WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-water_quality'
METATAB_DOC='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-water_quality/metadata.csv'
METATAB_WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-water_quality'
METATAB_PACKAGE='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-water_quality/'
name='tides_river_rain'
description='Combined max, min tides, San Diego river flow and daily precipitation'
url='notebooks/Combine.ipynb#comb'

In [3]:
METAPACK_BUILDING=True

In [4]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from metapack.appurl import SearchUrl 
SearchUrl.initialize()

print(np.__version__)



1.14.5


In [5]:
pkg = mp.jupyter.open_source_package()
pkg

In [6]:
r =  pkg.reference('la_jolla_tide')
r

Header,Type,Description
date_time,datetime,
water_level,number,"Water level, relative to datum, in feet"
type,string,"HH Daily high, H High, LL Daily Low, L Low"
I,integer,
L,integer,


In [7]:
water_levels = r.read_csv(parse_dates=True)
wl_by_day = water_levels.set_index(['date_time','type']).water_level.unstack(1).resample('1d').max()
wl_by_day.columns = ['wl_high_1', 'wl_high_2', 'wl_low_1','wl_low_2']
wl_by_day['wl_max'] = np.maximum(wl_by_day.wl_high_1, wl_by_day.wl_high_2 )
wl_by_day['wl_min'] = np.maximum(wl_by_day.wl_low_1, wl_by_day.wl_low_2 )
wl_by_day = wl_by_day[['wl_max','wl_min']]
wl_by_day.head()

,wl_max,wl_min
date_time,,
2000-01-01,5.043,1.546
2000-01-02,4.974,1.644
2000-01-03,5.234,1.651
2000-01-04,5.270,1.549
2000-01-05,5.617,1.792


In [8]:
dist = pkg.reference('sd_river_discharge').read_csv(parse_dates=True).set_index('datetime').resample('1d').mean()
dist = dist[['discharge_rate']].copy()
dist.head()

,discharge_rate
datetime,
1988-01-01,22.0
1988-01-02,20.0
1988-01-03,18.0
1988-01-04,18.0
1988-01-05,21.0


In [9]:
usecols = ['date', 'reporttpye', 'dailyprecip', 'hourlyprecip']
lcd = pkg.reference('local_climate').read_csv(parse_dates=True, usecols=usecols) #.set_index('datetime').resample('1d').mean()

lcd.rename(columns={'reporttpye': 'reporttype'}, inplace=True)
lcd = lcd[lcd.reporttype=='FM-15'].copy()
lcd.fillna(0, inplace=True)
lcd['date'] = lcd.date.dt.date
lcd = lcd.groupby('date').dailyprecip.max()


In [10]:
comb = dist.join(lcd).join(wl_by_day)
comb.head()

,discharge_rate,dailyprecip,wl_max,wl_min
datetime,,,,
1988-01-01,22.0,NaN,NaN,NaN
1988-01-02,20.0,NaN,NaN,NaN
1988-01-03,18.0,NaN,NaN,NaN
1988-01-04,18.0,NaN,NaN,NaN
1988-01-05,21.0,NaN,NaN,NaN


In [11]:
comb.corr()

,discharge_rate,dailyprecip,wl_max,wl_min
discharge_rate,1.000000,0.637036,0.045262,-0.001102
dailyprecip,0.637036,1.000000,0.050992,0.012542
wl_max,0.045262,0.050992,1.000000,-0.155287
wl_min,-0.001102,0.012542,-0.155287,1.000000


In [12]:

_1 = comb.rolling('3d').sum()

comb['recent_precip'] = _1.dailyprecip.round(2)
comb['recent_total_discharge'] = _1.discharge_rate.round(2)

_2 = _1.rolling('3d').max()

comb['recent_max_discharge'] = _2.discharge_rate.round(2)

comb[comb.recent_precip > 0].head(20)

,discharge_rate,dailyprecip,wl_max,wl_min,recent_precip,recent_total_discharge,recent_max_discharge
datetime,,,,,,,
2008-10-05,2.46,0.18,NaN,NaN,0.18,5.20,5.20
2008-10-06,2.78,0.00,NaN,NaN,0.18,6.70,6.70
2008-10-07,2.01,0.00,NaN,NaN,0.18,7.25,7.25
2008-11-05,16.00,0.14,4.036,3.268,0.14,32.97,32.97
2008-11-06,4.74,0.00,3.810,3.154,0.14,36.64,36.64
2008-11-07,2.75,0.00,NaN,2.606,0.14,23.49,36.64
2008-11-26,231.00,0.99,6.192,2.350,0.99,233.79,233.79
2008-11-27,270.00,0.10,6.290,2.573,1.09,502.44,502.44
2008-11-28,247.00,1.26,5.952,2.438,2.35,748.00,748.00


In [13]:
comb = comb[['discharge_rate', 'recent_total_discharge', 'recent_max_discharge',
             'dailyprecip', 'recent_precip', 'wl_max', 'wl_min']]

In [14]:
print(list(comb.columns))

['discharge_rate', 'recent_total_discharge', 'recent_max_discharge', 'dailyprecip', 'recent_precip', 'wl_max', 'wl_min']


In [15]:
%mt_materialize comb '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-4' 

{
    "df_name": "comb",
    "path": "/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-4/comb.csv"
}


In [16]:

%mt_materialize_all '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-water_quality-4'


[]


In [17]:

%mt_show_metatab



Declare: metatab-latest
Section: Resources


In [18]:

%mt_show_libdirs



[]
